In [1]:
# notebooks/3c_extract_pdf_pairs_with_llm.ipynb

# ========================================
# Cell 1: 라이브러리 import 및 설정
# ========================================

import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('../src'))

from processors.pdf_text_extractor import PDFTextExtractor
from processors.pdf_pair_matcher import PDFPairMatcher
from processors.llm_pdf_tm_generator import LLMPDFTMGenerator  # 🆕 새로운 LLM 생성기
from config import PROCESSED_DIR, PDF_DOWNLOAD_DIR

# 처리할 언어 쌍들 정의
LANGUAGE_PAIRS = [
    ('en', 'ko'),  # 영어-한국어
    ('en', 'ja'),  # 영어-일본어
]

# 필요한 디렉토리 생성
os.makedirs(PROCESSED_DIR, exist_ok=True)

print(f"🤖 LLM 기반 PDF 번역쌍 추출 시작")
print(f"🌐 처리할 언어 쌍: {LANGUAGE_PAIRS}")
print(f"📁 PDF 폴더: {PDF_DOWNLOAD_DIR}")
print(f"📁 결과 저장 폴더: {PROCESSED_DIR}")

# PDF 폴더 존재 확인
for lang_pair in LANGUAGE_PAIRS:
    for lang in lang_pair:
        lang_dir = os.path.join(PDF_DOWNLOAD_DIR, lang)
        if os.path.exists(lang_dir):
            pdf_count = len([f for f in os.listdir(lang_dir) if f.endswith('.pdf')])
            print(f"   - {lang.upper()}: {pdf_count}개 PDF")
        else:
            print(f"   - {lang.upper()}: ❌ 폴더 없음 ({lang_dir})")



🤖 LLM 기반 PDF 번역쌍 추출 시작
🌐 처리할 언어 쌍: [('en', 'ko'), ('en', 'ja')]
📁 PDF 폴더: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/2_downloaded/pdfs
📁 결과 저장 폴더: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed
   - EN: 101개 PDF
   - KO: ❌ 폴더 없음 (/mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/2_downloaded/pdfs/ko)
   - EN: 101개 PDF
   - JA: 95개 PDF


In [ ]:
# ========================================
# Cell 2: LLM 기반 처리 (개선된 버전)
# ========================================

# 🆕 LLM 기반 생성기 초기화
llm_generator = LLMPDFTMGenerator(fast_mode=True)
basic_extractor = PDFTextExtractor(min_text_length=25)

all_results = {}

for source_lang, target_lang in LANGUAGE_PAIRS:
    print(f"\n{'='*80}")
    print(f"🔄 {source_lang.upper()}-{target_lang.upper()} 언어 쌍 처리 시작 (LLM 기반)")
    print(f"{'='*80}")
    
    # 1. PDF 페어 찾기
    matcher = PDFPairMatcher(source_lang, target_lang)
    pdf_pairs = matcher.find_pdf_pairs()
    
    if not pdf_pairs:
        print(f"❌ {source_lang.upper()}-{target_lang.upper()} PDF 페어를 찾을 수 없습니다.")
        continue
    
    # 2. LLM 기반 번역 쌍 생성
    pairs_for_this_language = []
    
    # 🔧 일부만 처리하여 테스트 (전체 처리는 시간이 오래 걸림)
    test_pairs = pdf_pairs[:]  # 처음 5개만 테스트
    
    for i, (source_path, target_path) in enumerate(test_pairs):
        print(f"\n[{i+1}/{len(test_pairs)}] 📄 처리 중: {os.path.basename(target_path)}")
        
        try:
            # 🆕 기본 텍스트 추출 (수정된 방식)
            import fitz
            
            # Source PDF 텍스트 추출
            source_raw_text = ""
            with fitz.open(source_path) as doc:
                for page in doc:
                    source_raw_text += page.get_text() + "\n\n"
            
            # Target PDF 텍스트 추출  
            target_raw_text = ""
            with fitz.open(target_path) as doc:
                for page in doc:
                    target_raw_text += page.get_text() + "\n\n"
            
            print(f"   - 원본 텍스트 추출 완료 ({len(source_raw_text)} / {len(target_raw_text)} 문자)")
            
            # 🆕 LLM으로 의미있는 세그먼트 생성
            print(f"   - 🤖 {source_lang.upper()} LLM 세그먼트 생성 중...")
            source_segments = llm_generator.generate_meaningful_segments(source_raw_text, source_lang, target_lang)
            
            print(f"   - 🤖 {target_lang.upper()} LLM 세그먼트 생성 중...")
            target_segments = llm_generator.generate_meaningful_segments(target_raw_text, target_lang, source_lang)
            
            print(f"   - LLM 세그먼트: {source_lang.upper()} {len(source_segments)}개, {target_lang.upper()} {len(target_segments)}개")

            # 함수 호출 전에 디버깅 출력 추가
            print(f"   - 함수 호출 인자: source_segments({len(source_segments)}), target_segments({len(target_segments)})")
            print(f"   - 경로: {source_path}, {target_path}")
            print(f"   - 언어: {source_lang}, {target_lang}")

            
            # 🆕 문맥 정보를 포함한 번역 쌍 생성
            pairs = llm_generator.create_translation_pairs_with_context(
                source_segments, target_segments, 
                source_path, target_path,  # 🆕 source_path 추가
                source_lang, target_lang
            )
            pairs_for_this_language.extend(pairs)
            
            print(f"   - ✅ {len(pairs)}개 고품질 번역 쌍 생성")
            
            # 샘플 결과 출력
            if pairs:
                sample = pairs[0]
                print(f"   - 🔍 샘플 (유사도: {sample['similarity_score']:.3f}):")
                print(f"     {source_lang.upper()}: {sample['source_text'][:60]}...")
                print(f"     {target_lang.upper()}: {sample['target_text'][:60]}...")
                print(f"     카테고리: {sample.get('source_category', 'N/A')} → {sample.get('target_category', 'N/A')}")
            
        except Exception as e:
            print(f"   - ❌ 오류: {e}")
    
    all_results[f"{source_lang}_{target_lang}"] = pairs_for_this_language
    print(f"\n🎉 {source_lang.upper()}-{target_lang.upper()}: 총 {len(pairs_for_this_language)}개 고품질 번역 쌍 완료!")




🔄 EN-KO 언어 쌍 처리 시작 (LLM 기반)
❌ 매칭 테이블을 찾을 수 없습니다: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/1_input/pdf_list_master_en_ko.csv
❌ EN-KO PDF 페어를 찾을 수 없습니다.

🔄 EN-JA 언어 쌍 처리 시작 (LLM 기반)
📋 EN-JA 매칭 테이블에서 101개 항목을 로드했습니다.
✅ 실제 존재하는 100개의 EN-JA PDF 페어를 발견했습니다.


NameError: name 'test_pairs' is not defined

In [ ]:
# ========================================
# Cell 3: 결과 비교 및 저장
# ========================================

total_pairs = 0

print(f"\n{'='*80}")
print("📊 LLM 기반 번역쌍 추출 결과")
print(f"{'='*80}")

for lang_pair, pairs in all_results.items():
    if pairs:
        # 🆕 향상된 결과 파일 저장
        csv_path = os.path.join(PROCESSED_DIR, f"pdf_translation_pairs_{lang_pair}_llm_enhanced.csv")
        df = pd.DataFrame(pairs)
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        
        total_pairs += len(pairs)
        source_lang, target_lang = lang_pair.split('_')
        
        print(f"\n🌐 {source_lang.upper()}-{target_lang.upper()}:")
        print(f"   💾 저장: {os.path.basename(csv_path)}")
        print(f"   📄 번역 쌍: {len(pairs)}개")
        print(f"   📊 평균 유사도: {df['similarity_score'].mean():.3f}")
        print(f"   🎯 평균 신뢰도: {df['source_confidence'].mean():.3f}")
        
        # 🆕 카테고리별 분포
        if 'source_category' in df.columns:
            category_dist = df['source_category'].value_counts()
            print(f"   📋 카테고리 분포: {dict(category_dist)}")

# 🆕 품질 향상 분석
if total_pairs > 0:
    print(f"\n🎉 LLM 기반 고품질 번역쌍 추출 완료!")
    print(f"   📈 총 번역 쌍: {total_pairs}개")
    print(f"   🚀 기존 대비 예상 품질 향상: 높은 문맥 보존, 의미 단위 완성도")
    print(f"   💡 다음 단계: 이 데이터를 base_tm.csv와 통합하여 최종 TM 구축")
else:
    print(f"\n⚠️ 생성된 번역 쌍이 없습니다. 설정을 확인해주세요.")

In [ ]:
# ========================================
# Cell 4: 결과 저장 및 분석
# ========================================

import pandas as pd
from datetime import datetime

total_pairs = 0
all_pairs_combined = []

print(f"\n{'='*80}")
print("📊 LLM 기반 번역쌍 추출 결과 저장")
print(f"{'='*80}")

for lang_pair, pairs in all_results.items():
    if pairs:
        source_lang, target_lang = lang_pair.split('_')
        
        # CSV 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        csv_filename = f"pdf_tm_{lang_pair}_llm_enhanced_{timestamp}.csv"
        csv_path = os.path.join(PROCESSED_DIR, csv_filename)
        
        df = pd.DataFrame(pairs)
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        
        total_pairs += len(pairs)
        all_pairs_combined.extend(pairs)
        
        print(f"\n🌐 {source_lang.upper()}-{target_lang.upper()}:")
        print(f"   💾 저장: {csv_filename}")
        print(f"   📄 번역 쌍: {len(pairs)}개")
        print(f"   📊 평균 유사도: {df['similarity_score'].mean():.3f}")

# 통합 파일 저장
# if total_pairs > 0:
    # combined_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # combined_filename = f"pdf_tm_all_languages_llm_enhanced_{combined_timestamp}.csv"
    # combined_csv_path = os.path.join(PROCESSED_DIR, combined_filename)
    
    # combined_df = pd.DataFrame(all_pairs_combined)
    # combined_df.to_csv(combined_csv_path, index=False, encoding='utf-8-sig')
    
    # print(f"\n📦 통합 파일: {combined_filename}")
    # print(f"   📊 전체 번역 쌍: {len(all_pairs_combined)}개")
    # print(f"   🌐 지원 언어: {combined_df['target_lang'].unique().tolist()}")

print(f"\n✨ CSV 저장 완료!")